# SCC02713 - Introdução à Bioinformática
## Trabalho Prático: Classificação de Elementos Transponíveis
### Alunos:
Marcos Paulo Rocha Baltazar - 123456
Rebeca Vieira Carvalho - 12543530
Yvis Freire Silva Santos - 12608793

#### Instruções:
- Criar pasta "gff3_files" no ambiente e subir os arquivos correspondentes.




In [ ]:
!pip install biopython wget
!git clone https://github.com/Bonidia/MathFeature.git MathFeature
%cd MathFeature
!pip install -r requirements.txt
!apt-get -y install python3-igraph
%cd ..

fatal: destination path 'MathFeature' already exists and is not an empty directory.
/content/MathFeature
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-igraph is already the newest version (0.9.9-1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
/content


In [ ]:
import os
import wget
import pandas as pd
from Bio import Entrez, SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [ ]:
classes_te = {
    'LTRS': 'TEAnnotationFinal_LTR.gff3',
    'LINES': 'TEAnnotationFinal_LINE.gff3',
    'SINES': 'TEAnnotationFinal_SINE.gff3',
    'TIRS': 'TEAnnotationFinal_TIR.gff3',
    'MITES': 'TEAnnotationFinal_MITE.gff3',
    'Helitrons': 'TEAnnotationFinal_Helitron.gff3'
}

url = 'http://apte.cp.utfpr.edu.br/te-annotation/zea_mays/'

gff_dir = 'gff3_files/'
os.makedirs(gff_dir, exist_ok=True)

for classe, filename in classes_te.items():
    if not os.path.exists(gff_dir + filename):
        wget.download(url + filename, out=gff_dir)

print("Arquivos .gff3 baixados.")

Arquivos .gff3 baixados.


In [ ]:
col_names = ['Chr', 'Source', 'Class', 'Start', 'End', 'Score', 'Strand', 'Phase', 'Attributes']

all_te_locations = []

print("Iniciando processamento dos arquivos GFF3...")

for classe, filename in classes_te.items():
    file_path = os.path.join(gff_dir, filename)

    try:
        df = pd.read_csv(
            file_path,
            sep='\t',
            comment='#',
            header=None,  # O arquivo não tem cabeçalho
            names=col_names
        )

        # 1. Filtrar linhas com Strand positivo (+)
        # Usamos .astype(str) para garantir que a coluna 'Strand' seja comparada como texto
        df_filtered = df[df['Strand'].astype(str).str.strip() == '+'].copy()

        # 2. Recuperar Chr, Start e End
        # E garantir que Start e End sejam números
        df_filtered = df_filtered[['Chr', 'Start', 'End']]
        df_filtered['Start'] = pd.to_numeric(df_filtered['Start'])
        df_filtered['End'] = pd.to_numeric(df_filtered['End'])

        # 3. Adicionar a coluna da Classe (que é o nome do arquivo)
        df_filtered['Classe'] = classe

        all_te_locations.append(df_filtered)
        print(f"  - Processado {filename}: {len(df_filtered)} TEs com Strand (+)")

    except FileNotFoundError:
        print(f"ERRO: Arquivo {file_path} não encontrado. Certifique-se de fazer o upload.")
    except Exception as e:
        print(f"Erro ao processar {filename}: {e}")

# Consolida todos os TEs em um único DataFrame
if all_te_locations:
    df_locations = pd.concat(all_te_locations, ignore_index=True)
    print(f"\nProcessamento concluído. Total de TEs a extrair: {len(df_locations)}")
    print(df_locations.head())
else:
    print("\nNenhum arquivo GFF processado. Verifique os uploads e nomes dos arquivos.")

Iniciando processamento dos arquivos GFF3...
  - Processado TEAnnotationFinal_LTR.gff3: 191815 TEs com Strand (+)
  - Processado TEAnnotationFinal_LINE.gff3: 11174 TEs com Strand (+)
  - Processado TEAnnotationFinal_SINE.gff3: 3755 TEs com Strand (+)
  - Processado TEAnnotationFinal_TIR.gff3: 83849 TEs com Strand (+)
  - Processado TEAnnotationFinal_MITE.gff3: 26350 TEs com Strand (+)
  - Processado TEAnnotationFinal_Helitron.gff3: 24769 TEs com Strand (+)

Processamento concluído. Total de TEs a extrair: 341712
  Chr      Start        End Classe
0   4  122450121  122450632   LTRS
1   5    4778869    4779015   LTRS
2   2  197296548  197297070   LTRS
3   9   89034632   89035827   LTRS
4   9   39829863   39830128   LTRS


In [ ]:
# Download do Genoma via NCBI

Entrez.email = "yvis_freire@usp.br"

accession_numbers = {
    '1': 'LR618874.1',
    '2': 'LR618875.1',
    '3': 'LR618876.1',
    '4': 'LR618877.1',
    '5': 'LR618878.1',
    '6': 'LR618879.1',
    '7': 'LR618880.1',
    '8': 'LR618881.1',
    '9': 'LR618882.1',
    '10': 'LR618883.1',
    'MT': 'AY506529.1',
    'Pitd': 'X86563.2'
}

fasta_dir = 'fasta_files/'
os.makedirs(fasta_dir, exist_ok=True)

print("Baixando sequências do genoma...")

genomes = {}

for chr_name, acc_num in accession_numbers.items():
    fasta_path = os.path.join(fasta_dir, f"chr_{chr_name}.fasta")

    if not os.path.exists(fasta_path):
        try:
            handle = Entrez.efetch(db="nucleotide", id=acc_num, rettype="fasta", retmode="text")
            fasta_data = handle.read()
            handle.close()

            with open(fasta_path, "w") as f:
                f.write(fasta_data)
            print(f"  - Cromossomo {chr_name} ({acc_num}) baixado.")

        except Exception as e:
            print(f"Erro ao baixar Chr {chr_name}: {e}")
    else:
        print(f"  - Cromossomo {chr_name} já existe, pulando download.")

    try:
        record = SeqIO.read(fasta_path, "fasta")
        genomes[chr_name] = record.seq
    except Exception as e:
        print(f"Erro ao ler FASTA para Chr {chr_name}: {e}")

print("\nSequências do genoma carregadas na memória.")

Baixando sequências do genoma...
  - Cromossomo 1 (LR618874.1) baixado.
  - Cromossomo 2 (LR618875.1) baixado.
  - Cromossomo 3 (LR618876.1) baixado.
  - Cromossomo 4 (LR618877.1) baixado.
  - Cromossomo 5 (LR618878.1) baixado.
  - Cromossomo 6 (LR618879.1) baixado.
  - Cromossomo 7 (LR618880.1) baixado.
  - Cromossomo 8 (LR618881.1) baixado.
  - Cromossomo 9 (LR618882.1) baixado.
  - Cromossomo 10 (LR618883.1) baixado.
  - Cromossomo MT (AY506529.1) baixado.
  - Cromossomo Pitd (X86563.2) baixado.

Sequências do genoma carregadas na memória.


In [ ]:
# Extração das Sequências

print("Iniciando extração das sequências de TEs...")

seq_dataset = []

for index, row in df_locations.iterrows():
    chr_name = str(row['Chr'])
    start = int(row['Start'])
    end = int(row['End'])
    classe = row['Classe']

    chromosome_seq = genomes.get(chr_name)

    if chromosome_seq:
        # Ajuste de índice: GFF (1-based) para Python (0-based)
        start_idx = start - 1
        end_idx = end

        te_seq = chromosome_seq[start_idx:end_idx]

        seq_dataset.append({
            'Cromossomo': chr_name,
            'Sequencia': str(te_seq),
            'Classe': classe
        })

df_final = pd.DataFrame(seq_dataset)

if df_final.empty:
    print("ERRO: Nenhum dado de TE foi extraído.")
else:
    print(f"\nExtração concluída. Total de sequências: {len(df_final)}")
    print("Dataset final (amostra):")
    print(df_final.head())

    df_final.to_csv('te_dataset_final.csv', index=False)

Iniciando extração das sequências de TEs...

Extração concluída. Total de sequências: 338334
Dataset final (amostra):
  Cromossomo                                          Sequencia Classe
0          4  TCACAAAGTGGATAGAATACAAGCACATCACCTCTTTGACCTCAGC...   LTRS
1          5  ACCAATTGTGTGGTGGCCCTTGCGGGGAAGTTTTGTTCCCGGCTTT...   LTRS
2          2  GCCGTTGGCGCGACGCGGGGCGCGCGCGGGCGTGCTCTGGCGAGCT...   LTRS
3          9  GTCCGGGGGACCGTTGTCCGCCCCGAGGCTATTTTAGCCAGCTCAT...   LTRS
4          9  GAAGCAGCTCGTGGAGATGCTCAAGCGGTCACCCTACTTCCTCTCC...   LTRS
